In [1]:
import os
import gradio as gr
import openai
from gtts import gTTS
import tempfile
from dotenv import load_dotenv
import speech_recognition as sr
import io

d:\kpr-hackathon\chat\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [3]:
client = openai.OpenAI()
MODEL = "gpt-4o-mini"

In [4]:
SYSTEM_MESSAGE = """
You are an advanced EV Battery & Vehicle Assistant specialized in answering queries about electric vehicles (EVs) in India, including brands like Tata, Mahindra, MG, Hyundai, Kia, BYD, and others.
Your purpose is to provide accurate, clear, and useful information based only on official EV manuals, technical documentation, verified specifications, and authentic sources.

Guidelines:

Cover all aspects of EVs in India: battery (SoH, RUL, charging cycles, warnings, troubleshooting), vehicle performance, charging infrastructure, warranty, and usage best practices.

When answering, always use simple, human-like language and avoid unnecessary jargon.

If a query is vague, interpret it intelligently and provide the most relevant and helpful explanation instead of rejecting it.

Stay factual and grounded in available manuals, datasets, or trusted references—never invent details.

Structure answers in a way that is easy to follow, with step-by-step explanations when needed.

Response Format:
Answer: [Clear and concise explanation]
Reference: [Mention the manual/section/source if available]
"""


In [6]:
def chatbot_text(user_input, history):
    messages = [{"role": "system", "content": SYSTEM_MESSAGE}]
    for user, bot in history:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": bot})

    messages.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    answer = response.choices[0].message.content

    history.append((user_input, answer))
    return history, history, ""

In [ ]:
with gr.Blocks(
    theme=gr.themes.Base(),
    title="Flexi-EV Assistant",
    css="""
    /* Dark Theme Variables */
    :root {
        --primary-dark: #00403C;
        --primary-medium: #005951;
        --primary-light: #8FD6C2;
        --bg-dark: #001A18;
        --bg-medium: #002B27;
        --text-primary: #FFFFFF;
        --text-secondary: #B0B0B0;
        --accent: #8FD6C2;
    }
    
    /* Global Styling - Mobile First */
    .gradio-container {
        background: var(--bg-dark) !important;
        color: var(--text-primary) !important;
        font-family: 'Inter', 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif !important;
        width: 100% !important;
        max-width: 100vw !important;
        margin: 0 !important;
        padding: 10px !important;
        min-height: 100vh !important;
        box-sizing: border-box !important;
    }
    
    /* Mobile Responsive Container */
    @media (min-width: 768px) {
        .gradio-container {
            max-width: 900px !important;
            margin: 0 auto !important;
            padding: 20px !important;
        }
    }
    
    /* Header Styling - Mobile Optimized */
    .flexi-header {
        background: linear-gradient(135deg, var(--primary-dark) 0%, var(--primary-medium) 100%);
        padding: 20px 15px;
        border-radius: 15px;
        text-align: center;
        margin-bottom: 15px;
        box-shadow: 0 4px 20px rgba(0, 64, 60, 0.3);
    }
    
    .flexi-header h1 {
        color: var(--text-primary);
        font-size: 1.8em;
        font-weight: 600;
        margin: 0;
        letter-spacing: -0.3px;
    }
    
    .flexi-header p {
        color: var(--accent);
        font-size: 0.95em;
        margin: 8px 0 0 0;
        opacity: 0.9;
        line-height: 1.4;
    }
    
    /* Desktop Header */
    @media (min-width: 768px) {
        .flexi-header {
            padding: 25px;
            border-radius: 20px;
            margin-bottom: 30px;
        }
        
        .flexi-header h1 {
            font-size: 2.2em;
        }
        
        .flexi-header p {
            font-size: 1.1em;
        }
    }
    
    /* Chat Container - Mobile First */
    .chat-container {
        background: var(--bg-medium) !important;
        border: 1px solid var(--primary-medium) !important;
        border-radius: 15px !important;
        min-height: 400px !important;
        max-height: 60vh !important;
        padding: 15px !important;
        box-shadow: 0 8px 32px rgba(0, 0, 0, 0.4) !important;
        overflow-y: auto !important;
    }
    
    /* Desktop Chat Container */
    @media (min-width: 768px) {
        .chat-container {
            border-radius: 20px !important;
            min-height: 550px !important;
            max-height: none !important;
            padding: 20px !important;
        }
    }
    
    /* Message Bubbles - Mobile Optimized */
    .message {
        margin: 10px 0;
        max-width: 85%;
        animation: fadeIn 0.3s ease-in;
        word-wrap: break-word;
    }
    
    .user-message {
        background: var(--primary-dark);
        color: var(--text-primary);
        margin-left: auto;
        border-radius: 15px 15px 3px 15px;
        padding: 10px 14px;
        box-shadow: 0 2px 8px rgba(0, 64, 60, 0.3);
        font-size: 0.9em;
        line-height: 1.4;
    }
    
    .bot-message {
        background: var(--primary-medium);
        color: var(--accent);
        margin-right: auto;
        border-radius: 15px 15px 15px 3px;
        padding: 10px 14px;
        box-shadow: 0 2px 8px rgba(0, 0, 0, 0.2);
        font-size: 0.9em;
        line-height: 1.4;
    }
    
    /* Desktop Message Bubbles */
    @media (min-width: 768px) {
        .message {
            max-width: 75%;
        }
        
        .user-message, .bot-message {
            border-radius: 18px 18px 4px 18px;
            padding: 12px 18px;
            font-size: 1em;
        }
        
        .bot-message {
            border-radius: 18px 18px 18px 4px;
        }
    }
    
    /* Input Area - Mobile First */
    .input-container {
        background: var(--bg-medium) !important;
        border: 1px solid var(--primary-medium) !important;
        border-radius: 15px !important;
        padding: 12px !important;
        margin-top: 15px !important;
        position: sticky !important;
        bottom: 10px !important;
        z-index: 100 !important;
    }
    
    .input-container textarea {
        background: var(--primary-dark) !important;
        border: 1px solid var(--primary-medium) !important;
        border-radius: 20px !important;
        color: var(--text-primary) !important;
        padding: 12px 16px !important;
        font-size: 16px !important; /* Prevents zoom on iOS */
        resize: none !important;
        width: 100% !important;
        box-sizing: border-box !important;
    }
    
    .input-container textarea::placeholder {
        color: var(--text-secondary) !important;
        opacity: 0.7 !important;
    }
    
    .input-container textarea:focus {
        border-color: var(--accent) !important;
        box-shadow: 0 0 0 2px rgba(143, 214, 194, 0.2) !important;
        outline: none !important;
    }
    
    /* Desktop Input */
    @media (min-width: 768px) {
        .input-container {
            border-radius: 20px !important;
            padding: 20px !important;
            margin-top: 20px !important;
            position: relative !important;
            bottom: auto !important;
        }
        
        .input-container textarea {
            border-radius: 25px !important;
            padding: 15px 20px !important;
            font-size: 1em !important;
        }
    }
    
    /* Send Button - Mobile Optimized */
    .send-button {
        background: linear-gradient(135deg, var(--accent) 0%, var(--primary-light) 100%) !important;
        border: none !important;
        border-radius: 50% !important;
        width: 45px !important;
        height: 45px !important;
        color: var(--primary-dark) !important;
        font-weight: bold !important;
        cursor: pointer !important;
        transition: all 0.3s ease !important;
        box-shadow: 0 3px 12px rgba(143, 214, 194, 0.3) !important;
        font-size: 1.2em !important;
        flex-shrink: 0 !important;
        margin-left: 10px !important;
    }
    
    .send-button:hover, .send-button:active {
        transform: translateY(-1px) !important;
        box-shadow: 0 4px 16px rgba(143, 214, 194, 0.4) !important;
    }
    
    /* Desktop Send Button */
    @media (min-width: 768px) {
        .send-button {
            width: 50px !important;
            height: 50px !important;
            margin-left: 15px !important;
        }
        
        .send-button:hover {
            transform: translateY(-2px) !important;
            box-shadow: 0 6px 20px rgba(143, 214, 194, 0.4) !important;
        }
    }
    
    /* Clear Button - Mobile Optimized */
    .clear-button {
        background: var(--primary-medium) !important;
        border: 1px solid var(--primary-light) !important;
        border-radius: 20px !important;
        color: var(--accent) !important;
        padding: 10px 20px !important;
        font-weight: 500 !important;
        transition: all 0.3s ease !important;
        width: 100% !important;
        margin-top: 10px !important;
        font-size: 0.9em !important;
    }
    
    .clear-button:hover, .clear-button:active {
        background: var(--primary-light) !important;
        color: var(--primary-dark) !important;
    }
    
    /* Desktop Clear Button */
    @media (min-width: 768px) {
        .clear-button {
            border-radius: 25px !important;
            padding: 12px 24px !important;
            width: auto !important;
            font-size: 1em !important;
        }
    }
    
    /* Mobile Input Row Layout */
    .input-row {
        display: flex !important;
        align-items: flex-end !important;
        gap: 0 !important;
        width: 100% !important;
    }
    
    .input-text-col {
        flex: 1 !important;
        min-width: 0 !important;
    }
    
    .input-button-col {
        flex-shrink: 0 !important;
        display: flex !important;
        align-items: flex-end !important;
    }
    
    /* Animations */
    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(10px); }
        to { opacity: 1; transform: translateY(0); }
    }
    
    @keyframes typing {
        0% { opacity: 0.3; }
        50% { opacity: 1; }
        100% { opacity: 0.3; }
    }
    
    .typing-indicator {
        animation: typing 1.5s infinite;
    }
    
    /* Mobile Scrollbar */
    ::-webkit-scrollbar {
        width: 6px;
    }
    
    ::-webkit-scrollbar-track {
        background: var(--bg-dark);
        border-radius: 3px;
    }
    
    ::-webkit-scrollbar-thumb {
        background: var(--primary-medium);
        border-radius: 3px;
    }
    
    ::-webkit-scrollbar-thumb:hover {
        background: var(--accent);
    }
    
    /* Desktop Scrollbar */
    @media (min-width: 768px) {
        ::-webkit-scrollbar {
            width: 8px;
        }
        
        ::-webkit-scrollbar-track, ::-webkit-scrollbar-thumb {
            border-radius: 4px;
        }
    }
    
    /* Labels and Text */
    label {
        color: var(--text-primary) !important;
        font-weight: 500 !important;
    }
    
    .gr-box {
        background: var(--bg-medium) !important;
        border-color: var(--primary-medium) !important;
    }
    
    /* Mobile Viewport Fix */
    @media (max-width: 767px) {
        .gradio-container {
            padding-bottom: 20px !important;
        }
        
        /* Fix for mobile keyboard */
        .gradio-container > div {
            min-height: auto !important;
        }
    }
    
    /* Touch Friendly Buttons */
    @media (hover: none) and (pointer: coarse) {
        .send-button, .clear-button {
            transform: none !important;
        }
        
        .send-button:active {
            transform: scale(0.95) !important;
        }
        
        .clear-button:active {
            transform: scale(0.98) !important;
        }
    }
    """
) as demo:
    
    # Header Section
    with gr.Row():
        gr.HTML("""
        <div class="flexi-header">
            <h1> Flexi-EV</h1>
            <p>Your intelligent EV companion for seamless electric mobility</p>
        </div>
        """)
    
    # Main Chat Interface
    with gr.Row():
        chatbot = gr.Chatbot(
            label="",
            height=400,  # Reduced for mobile
            show_copy_button=False,
            bubble_full_width=False,
            show_share_button=False,
            container=True,
            elem_classes=["chat-container"],
            avatar_images=(
                "https://cdn-icons-png.flaticon.com/512/1077/1077114.png",
                "https://cdn-icons-png.flaticon.com/512/4712/4712035.png"
            )
        )
    
    # Input Section - Mobile Optimized Layout
    with gr.Row(elem_classes=["input-row"]):
        with gr.Column(scale=1, elem_classes=["input-container", "input-text-col"]):
            text_input = gr.Textbox(
                label="",
                placeholder="Ask Flexi-EV...",
                lines=1,
                max_lines=3,
                show_label=False,
                container=False
            )
        
        with gr.Column(scale=0, min_width=60, elem_classes=["input-button-col"]):
            submit_btn = gr.Button(
                "→", 
                variant="primary",
                elem_classes=["send-button"]
            )
    
    # Control Buttons
    with gr.Row():
        with gr.Column():
            clear_btn = gr.Button(
                "Clear Conversation", 
                variant="secondary",
                elem_classes=["clear-button"],
                size="sm"
            )
    
    # State management
    state = gr.State([])
    
    # Event handlers with mobile-friendly interactions
    def handle_message(message, history):
        if not message.strip():
            return history, history, ""
        
        # Get actual response
        response_history, final_state, cleared_input = chatbot_text(message, history)
        
        return response_history, final_state, cleared_input
    
    submit_btn.click(
        handle_message,
        inputs=[text_input, state],
        outputs=[chatbot, state, text_input]
    )
    
    text_input.submit(
        handle_message,
        inputs=[text_input, state],
        outputs=[chatbot, state, text_input]
    )
    
    clear_btn.click(
        lambda: ([], []),
        outputs=[chatbot, state]
    )

demo.launch(share=True, show_error=True, inbrowser=True)

C:\Users\sajit\AppData\Local\Temp\ipykernel_11680\443470524.py:384: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
C:\Users\sajit\AppData\Local\Temp\ipykernel_11680\443470524.py:384: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://f7601f981d560eeeed.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
* Running on public URL: https://f7601f981d560eeeed.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


: 